In [3]:
### firstly open the file


import os

filepath  = r"C:\Users\Steve\OneDrive - University of Southampton\CulturalViolence\KnowledgeBases\data"

with open(os.path.join(filepath, "IHaveADream.txt"), 'r') as text:
    speech_text = text.readlines()

for line in speech_text[:5]:
    print(line)
    


Wall time: 0 ns
On February 24th, 1920, the first great mass meeting under the auspices of the new movement took place. In the Banquet Hall of the Hofbräuhaus in Munich the twenty-five theses which constituted the programme of our new party were expounded to an audience of nearly two thousand people and each thesis was enthusiastically received. 

Thus we brought to the knowledge of the public those first principles and lines of action along which the new struggle was to be conducted for the abolition of a confused mass of obsolete ideas and opinions which had obscure and often pernicious tendencies. A new force was to make its appearance among the timid and feckless bourgeoisie. This force was destined to impede the triumphant advance of the Marxists and bring the Chariot of Fate to a standstill just as it seemed about to reach its goal. 

It was evident that this new movement could gain the public significance and support which are necessary pre-requisites in such a gigantic struggle

In [8]:
%%time

## now we get the most positive and negative sentiment of the text
from textblob import TextBlob

counter = 0
most_pos_para = ''
most_neg_para = ''
most_pos_score = 0
most_neg_score = 0


for section in speech_text:
    paragraph = TextBlob(section)
    
    if paragraph.sentiment.polarity > most_pos_score:
        most_pos_para = paragraph
        most_pos_score = paragraph.sentiment.polarity
    elif paragraph.sentiment.polarity < most_neg_score:
        most_neg_para = paragraph
        most_neg_score = paragraph.sentiment.polarity
        
print('with a score of ', str(most_pos_score), ', the most positive paragraph is')
print(most_pos_para)
print()
print('with a score of ', str(most_neg_score), ', the most negative paragraph is')
print(most_neg_para)

with a score of  1.0 , the most positive paragraph is
A state which in this age of racial poisoning dedicates itself to the care of its best racial elements must some day become lord of the earth.


with a score of  -1.0 , the most negative paragraph is
Times of national collapse are determined by the preponderating influence of the worst elements. 

Wall time: 942 ms


In [10]:
## now we clean the document 

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from collections import Counter
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop]) ## remove stop words
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude) ## remove puncutation
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split()) ## lemmatize each word
    return normalized 
    

def Most_Common(lst):
    data = Counter(lst)
    return data.most_common(5)

doc_clean = [clean(doc).split() for doc in speech_text]
print('done')

### print the most common words from the first 5 lines.

for line in doc_clean[:5]:
    print(line)
    print(Most_Common(line)) 
    print(Counter(line).most_common(3))
    print('-----')

done
['february', '24th', '1920', 'first', 'great', 'mass', 'meeting', 'auspex', 'new', 'movement', 'took', 'place', 'banquet', 'hall', 'hofbräuhaus', 'munich', 'twentyfive', 'thesis', 'constituted', 'programme', 'new', 'party', 'expounded', 'audience', 'nearly', 'two', 'thousand', 'people', 'thesis', 'enthusiastically', 'received']
[('new', 2), ('thesis', 2), ('february', 1), ('24th', 1), ('1920', 1)]
[('new', 2), ('thesis', 2), ('february', 1)]
-----
['thus', 'brought', 'knowledge', 'public', 'first', 'principle', 'line', 'action', 'along', 'new', 'struggle', 'conducted', 'abolition', 'confused', 'mass', 'obsolete', 'idea', 'opinion', 'obscure', 'often', 'pernicious', 'tendency', 'new', 'force', 'make', 'appearance', 'among', 'timid', 'feckless', 'bourgeoisie', 'force', 'destined', 'impede', 'triumphant', 'advance', 'marxist', 'bring', 'chariot', 'fate', 'standstill', 'seemed', 'reach', 'goal']
[('new', 2), ('force', 2), ('thus', 1), ('brought', 1), ('knowledge', 1)]
[('new', 2), ('f

In [12]:
# Creating the term dictionary for the dataset, where every unique term is assigned an index. 

import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=5, num_words=5))

[(0, '0.009*"meeting" + 0.008*"one" + 0.007*"germany" + 0.006*"people" + 0.005*"would"'), (1, '0.015*"must" + 0.012*"state" + 0.009*"people" + 0.007*"national" + 0.007*"would"'), (2, '0.010*"would" + 0.009*"state" + 0.008*"people" + 0.006*"one" + 0.006*"german"')]


In [ ]:
%%time

### analyse the whole document to extract the topics and sentiment
# https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/
# https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

import os
from textblob import TextBlob
from collections import Counter
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
import collections
from gensim import corpora
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
Lda = gensim.models.ldamodel.LdaModel

### open file
filepath  = r"C:\Users\Steve\OneDrive - University of Southampton\CulturalViolence\KnowledgeBases\data"

with open(os.path.join(filepath, "MeinKampfv2.txt"), 'r') as text:
    speech_text = text.readlines()

# define common functions
def clean(doc): # text cleaning function
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop]) # remove stopwords
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude) # remove punctuation
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split()) #lemmatize words
    return normalized

def Most_Common(lst, quantity):
    data = Counter(lst)
    return data.most_common(quantity)

# speech pre-processing to a bag-of-words
i = TextBlob(''.join(speech_text))
doc_clean = [clean(doc).split() for doc in speech_text] # clean speech to a bag of words
dictionary = corpora.Dictionary(doc_clean) # Creating the term dictionary for the speech, where every unique term is assigned an index.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean] # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

print('Overall Text Topics: ', end='')
print(ldamodel.print_topics(num_topics=5, num_words=2))
print('-----')

print('Text Overall Sentiment: ', end='')
print(i.sentiment.polarity)
print('-----')

print('Text Common Nouns: ', end='')
print(Most_Common(i.noun_phrases, 15))
print('-----')

flat_list = []

for sublist in doc_clean:
    for item in sublist:
        flat_list.append(item)


print('Speech common words: ', end='')
print(Most_Common(flat_list, 5))
print('-----')

# generate speech summary data

counter = 0
most_pos_para = ''
most_neg_para = ''
pos_paras = []
neg_paras = []
most_pos_score = 0
most_neg_score = 0

for section in speech_text:
    paragraph = TextBlob(section)
    
    if paragraph.sentiment.polarity > most_pos_score and paragraph.sentiment.polarity < 1:
        most_pos_para = paragraph
        most_pos_score = paragraph.sentiment.polarity
    elif paragraph.sentiment.polarity < most_neg_score and paragraph.sentiment.polarity > -1:
        most_neg_para = paragraph
        most_neg_score = paragraph.sentiment.polarity
    elif paragraph.sentiment.polarity == 1:
        pos_paras.append(section)
    elif paragraph.sentiment.polarity == -1:
        neg_paras.append(section)
    
        
print('With a score of ', str(most_pos_score), ', the most positive paragraph is:')
print(most_pos_para)
print('-----')
print('With a score of ', str(most_neg_score), ', the most negative paragraph is:')
print(most_neg_para)
print('-----')
print('The paragraphs with a sentiment score of 1 are:')
for i in pos_paras:
    print(i)
print()
print('The paragraphs with a sentiment score of -1 are:')
for i in neg_paras:
    print(i)
print('-----')

# generate paragraph summary data by iterating through each paragraph

for section in speech_text:
    counter += 1
    paragraph = TextBlob(section)
    
    doc_clean = [clean(section).split()] # clean section
    dictionary = corpora.Dictionary(doc_clean) # Creating the term dictionary of each paragraph, where every unique term is assigned an index. 
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean] # Converting paragraph into Document Term Matrix using dictionary prepared above.
    if doc_term_matrix: ## skip over empty paragraphs
        ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
    
    print('Paragraph %d' %counter)
    print('    Paragraph topics: ', end='')
    print(ldamodel.print_topics(num_topics=5, num_words=2))
    print('    Paragraph sentiment: ', end='')
    print(paragraph.sentiment.polarity)
    print('    Paragraph nouns: ', end='')
    print(paragraph.noun_phrases)
    print('    Paragraph common words: ', end='')
    
    for w in doc_clean:
        print(Most_Common(w, 5))
        
    print()

Overall Text Topics: [(0, '0.009*"german" + 0.009*"people"'), (1, '0.009*"meeting" + 0.007*"one"'), (2, '0.013*"state" + 0.010*"would"')]
-----
Text Overall Sentiment: 0.0964078892045762
-----
Text Common Nouns: 